In [1]:
import numpy as np
from datetime import datetime, time
from sklearn.linear_model    import LinearRegression
from sklearn.preprocessing   import StandardScaler
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#import pandas_profiling

from catboost import CatBoostRegressor

from sklearn.model_selection import cross_val_score, KFold

from sklearn.metrics import mean_squared_error
#from xgboost import XGBRegressor
#from lightgbm import LGBMClassifier, cv, Dataset, train, plot_importance, LGBMRegressor
import plotly.express as px 
import plotly.graph_objects as go
from   plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import chart_studio.plotly as py
from   scipy.signal import correlate


%matplotlib inline
pd.options.display.max_rows = 1500

In [2]:
#init_notebook_mode(connected = False)

In [3]:
#pip install --user pandas-profiling
#pandas_profiling.ProfileReport(data)

In [4]:
def time_parse(df):

    month = {'JAN': '01', 
             'FEB': '02', 
             'MAR': '03', 
             'APR': '04',  
             'MAY': '05', 
             'JUN': '06', 
             'JUL': '07', 
             'AUG': '08', 
             'SEP': '09', 
             'OCT': '10', 
             'NOV': '11', 
             'DEC': '12' }

    Day   =        df.str[:2]
    Month =        df.str[2:5].map(month)
    Year  = ('20'+ df.str[5:7])
    Hour  =        df.str[8:10]
    Min   =        df.str[11:13]
    Sec   =        df.str[14:16]
    
    return pd.to_datetime( (Year  + Month + Day + Hour + Min + Sec) ,format='%Y%m%d%H%M%S')

#######################################################
#######################################################

def plot(x,y, title, xaxis, yaxis):
    target_data = go.Scatter(x=  x,
                             y=   y,
                             mode = 'lines',
                            )

    layout = go.Layout(title=title, xaxis=dict(title=xaxis),
                   yaxis=dict(title=yaxis),
                   )


    fig = go.Figure(data=[target_data], layout=layout)
    iplot(fig)

In [45]:
def data_load_transform(path):
    df = pd.read_csv(path)
    df = df.drop(['PERIOD'], axis=1)
    df.drop_duplicates(inplace=True)
    
    df['channel_type'].fillna('unknown', inplace=True) #Заменяем пропущенные значения на неизвестные
    #Обработка временных признаков
    df['TRDATETIME'] = time_parse(df['TRDATETIME'])   
    df['day'] =        (df['TRDATETIME'] - df['TRDATETIME'].min()).map(lambda x:x.days) 
    df['hour']        = df['TRDATETIME'].dt.hour
    df['weekday']     = df['TRDATETIME'].dt.dayofweek
    df['weekend']    = (df['weekday']>=5 ).astype(int) 
    df['month']       = df['TRDATETIME'].dt.month   
    df['month_start'] = (df['TRDATETIME'].dt.is_month_start).map({False:0, True:1}) 
    df['month_end'] =   (df['TRDATETIME'].dt.is_month_end).map({False:0, True:1}) 
    
    return df
   
data = data_load_transform('D:/regression.csv')

In [46]:
#Выбросы
data = data[(data['target_sum'] < data['target_sum'].quantile(0.99))]
data = data[(data['amount']     < data['amount'].quantile(0.99))]

In [48]:
print(data[data['target_flag']==0]['target_sum'].sum())
data = data[(data['target_sum'] > 0)]
data.drop('target_flag',axis=1,inplace=True)
#Находим target_sum только для активных клиентов

0.0


In [54]:
plot1 = data.groupby('cl_id',as_index=False)['target_sum'].mean()
plot2 = data.groupby('cl_id',as_index=False)['amount'].mean()
plot(plot1['cl_id'],plot1['target_sum'], 'Target Sum by Client Id', 'Client Id', 'Target Sum mean')
plot(plot2['cl_id'],plot2['amount'], 'Amount Sum by Client Id', 'Client Id', 'Amount Sum mean')

In [49]:
data.sort_values('cl_id' ,inplace=True)
#Сумма значений amount для пользователя
data['amount_sum'] = data.groupby('cl_id', as_index=False)['amount'].transform(lambda x: np.sum(x.values))

#Сумма по каждой из категорий транзакций
for trx_category in data['trx_category'].unique():
        data['sum_'+trx_category] = data[data['trx_category']==trx_category].groupby(['trx_category','cl_id'], as_index=False)['amount'].transform(
        lambda x: np.sum(x.values))

#Общее количество транзакций для каждого пользователя
data['trx_count'] = data.groupby('cl_id')['trx_category'].transform(lambda x: np.size(x.values)) 

#Разница в днях между первым и последним платежом
data['payday_diff'] = (data.groupby(['cl_id'])['day'].transform(lambda x: np.max(x.values)) - 
                       data.groupby(['cl_id'])['day'].transform(lambda x: np.min(x.values)))   

#Интенсивность платежа (общая сумма на разницу между первым и последним днём)
data['pay_intense'] = data['amount_sum']/data['payday_diff']

#Количество транзакций каждого типа для каждого пользователя
for trx_category in data['trx_category'].unique():
        data['count_'+trx_category] = data[data['trx_category'] == trx_category].groupby('cl_id')['amount'].transform(
        lambda x: np.size(x.values))

data.fillna(0, inplace=True) 

In [50]:
fig = px.imshow(data.corr(), title = 'Correlation Heatmap',width=800, height=800)
fig.show()

In [52]:
data.drop('TRDATETIME', axis=1, inplace=True)
df_with_dummies = pd.get_dummies(data, columns=['trx_category','channel_type'])
df_with_dummies.drop(['cl_id'],axis=1, inplace = True)
X = df_with_dummies.drop(['target_sum'], axis =1)
Y = df_with_dummies['target_sum']

In [53]:
kf = KFold(n_splits = 10, shuffle=True, random_state=42)

k = CatBoostRegressor(iterations=700, 
                      logging_level='Silent', learning_rate=0.01, l2_leaf_reg=6, max_depth = 10
                      )

score = np.mean(cross_val_score(
    k, X, np.log(Y+1), #target_sum в log масштабе
    scoring = 'neg_mean_squared_error', #отрицательный mse
    cv = kf, n_jobs = -1
))


print('RMSE mean = %.5f' % (-score)**0.5)
#print('R^2 = %.5f' % score)

RMSE mean = 1.26892
